In [8]:
import io

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload

import json
# from pdfminer.converter import TextConverter
# from pdfminer.pdfinterp import PDFPageInterpreter
# from pdfminer.pdfinterp import PDFResourceManager
# from pdfminer.pdfpage import PDFPage

In [9]:
from bs4 import BeautifulSoup
import os
import re
import requests
import string
import numpy as np
import pandas as pd
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [3]:
response = requests.get(
    url='https://proxy.scrapeops.io/v1/',
    params={
        'api_key': '378d2b00-d774-4869-bf97-325ac3d1de22',
        'url': 'https://www.al-akhbar.com/Editions',
    },
    )
soup = BeautifulSoup(response.text, 'html.parser')

In [10]:
container = soup.find('div', {'class': 'l-content row archive-issues'})
container = container.find('div', id='archive-days-wrap')

In [58]:
dic = {}

for a_tag in container.find_all('a', class_='day'):
    data_id = a_tag.get('data-id')
    link = a_tag.get('href')
    url = 'https://www.al-akhbar.com' + link + '/PDF'
    data_name = a_tag.get('data-oldtitle')
    date = link[10:].replace('/','-')
    dic[data_id] = [data_name,url,date]

In [70]:
df = pd.DataFrame(data=dic).T
df.columns = ['issue', 'url', 'date']
df = df[(df['date'] > '2010-01-01') & (df['date'] < '2020-01-01')]

In [7]:

def get_pdf(url):
    
    retries = Retry(total=5,
                status_forcelist=[429, 500, 502, 503, 504])
    response = requests.get(
    url='https://proxy.scrapeops.io/v1/',
    params={
        'api_key': '378d2b00-d774-4869-bf97-325ac3d1de22',
        'url': url,
    },
    )
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        container = soup.find('div', {'class': 'content'})
        pdf_link = container.find('a').get('href')
        pdf_link = 'https://www.al-akhbar.com' + pdf_link
        print('tamam')
        return pdf_link
    
    except:
        return None

In [196]:
mask = (df.pdf_link.isna())
df2 = df.loc[mask]
df2['pdf_link'] = df2['url'].apply(get_pdf)
df2.to_csv('new.csv')

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


/tmp/ipykernel_30697/1784432214.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pdf_link'] = df2['url'].apply(get_pdf)


In [203]:
mask3 = (df2.pdf_link.isna())
df3 = df2.loc[mask3]
df3

,issue,url,date,pdf_link
3474,العدد 3474,https://www.al-akhbar.com/Editions/2018/5/23/PDF,2018-5-23,None
3393,العدد 3393,https://www.al-akhbar.com/Editions/2018/2/12/PDF,2018-2-12,None
3307,العدد 3307,https://www.al-akhbar.com/Editions/2017/10/25/PDF,2017-10-25,None
3249,العدد 3249,https://www.al-akhbar.com/Editions/2017/8/12/PDF,2017-8-12,None
3123,العدد 3123,https://www.al-akhbar.com/Editions/2017/3/9/PDF,2017-3-9,None
...,...,...,...,...
1155,العدد 1155,https://www.al-akhbar.com/Editions/2010/6/30/PDF,2010-6-30,None
1038,العدد 1038,https://www.al-akhbar.com/Editions/2010/2/6/PDF,2010-2-6,None
1043,العدد 1043,https://www.al-akhbar.com/Editions/2010/2/13/PDF,2010-2-13,None
1017,العدد 1017,https://www.al-akhbar.com/Editions/2010/1/13/PDF,2010-1-13,None


In [204]:
df3['pdf_link'] = df3['url'].apply(get_pdf)
df3.to_csv('new2.csv')

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


/tmp/ipykernel_30697/3502492366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['pdf_link'] = df3['url'].apply(get_pdf)


In [4]:
df3 = pd.read_csv('new3.csv')
df3

,Unnamed: 0,issue,url,date,pdf_link
0,3393,العدد 3393,https://www.al-akhbar.com/Editions/2018/2/12/PDF,2018-2-12,NaN
1,1873,العدد 1873,https://www.al-akhbar.com/Editions/2012/12/4/PDF,2012-12-4,NaN
2,1893,العدد 1893,https://www.al-akhbar.com/Editions/2012/12/28/PDF,2012-12-28,NaN
3,1853,العدد 1853,https://www.al-akhbar.com/Editions/2012/11/7/PDF,2012-11-7,NaN
4,1858,العدد 1858,https://www.al-akhbar.com/Editions/2012/11/13/PDF,2012-11-13,NaN
5,1608,العدد 1608,https://www.al-akhbar.com/Editions/2012/11/13/PDF,2012-11-13,NaN
6,1820,العدد 1820,https://www.al-akhbar.com/Editions/2012/9/28/PDF,2012-9-28,NaN
7,1773,العدد 1773,https://www.al-akhbar.com/Editions/2012/8/2/PDF,2012-8-2,NaN
8,1791,العدد 1791,https://www.al-akhbar.com/Editions/2012/8/25/PDF,2012-8-25,NaN
9,1738,العدد 1738,https://www.al-akhbar.com/Editions/2012/6/22/PDF,2012-6-22,NaN


In [8]:
df3['pdf_link'] = df3['url'].apply(get_pdf)
df3

,Unnamed: 0,issue,url,date,pdf_link
0,3393,العدد 3393,https://www.al-akhbar.com/Editions/2018/2/12/PDF,2018-2-12,None
1,1873,العدد 1873,https://www.al-akhbar.com/Editions/2012/12/4/PDF,2012-12-4,None
2,1893,العدد 1893,https://www.al-akhbar.com/Editions/2012/12/28/PDF,2012-12-28,None
3,1853,العدد 1853,https://www.al-akhbar.com/Editions/2012/11/7/PDF,2012-11-7,None
4,1858,العدد 1858,https://www.al-akhbar.com/Editions/2012/11/13/PDF,2012-11-13,None
5,1608,العدد 1608,https://www.al-akhbar.com/Editions/2012/11/13/PDF,2012-11-13,None
6,1820,العدد 1820,https://www.al-akhbar.com/Editions/2012/9/28/PDF,2012-9-28,None
7,1773,العدد 1773,https://www.al-akhbar.com/Editions/2012/8/2/PDF,2012-8-2,None
8,1791,العدد 1791,https://www.al-akhbar.com/Editions/2012/8/25/PDF,2012-8-25,None
9,1738,العدد 1738,https://www.al-akhbar.com/Editions/2012/6/22/PDF,2012-6-22,None


In [79]:
df = pd.read_csv('pdf_df.csv')
df

,issue,url,date,pdf_link
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...
1,العدد 3923,https://www.al-akhbar.com/Editions/2019/12/3/PDF,2019-12-3,NaN
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...
...,...,...,...,...
2923,العدد 1028,https://www.al-akhbar.com/Editions/2010/1/26/PDF,2010-1-26,NaN
2924,العدد 1029,https://www.al-akhbar.com/Editions/2010/1/27/PDF,2010-1-27,NaN
2925,العدد 1030,https://www.al-akhbar.com/Editions/2010/1/28/PDF,2010-1-28,NaN
2926,العدد 1031,https://www.al-akhbar.com/Editions/2010/1/29/PDF,2010-1-29,NaN


In [80]:
df1 = pd.read_csv('new.csv')
df2 = pd.read_csv('new2.csv')
df3 = pd.read_csv('new3.csv')

In [5]:
df

,issue,url,date,pdf_link
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...
1,العدد 3923,https://www.al-akhbar.com/Editions/2019/12/3/PDF,2019-12-3,https://www.al-akhbar.com/PDF_Files/3885/alakh...
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...
...,...,...,...,...
2923,العدد 1028,https://www.al-akhbar.com/Editions/2010/1/26/PDF,2010-1-26,https://www.al-akhbar.com/PDF_Files/1028/alakh...
2924,العدد 1029,https://www.al-akhbar.com/Editions/2010/1/27/PDF,2010-1-27,https://www.al-akhbar.com/PDF_Files/1029/alakh...
2925,العدد 1030,https://www.al-akhbar.com/Editions/2010/1/28/PDF,2010-1-28,https://www.al-akhbar.com/PDF_Files/1030/alakh...
2926,العدد 1031,https://www.al-akhbar.com/Editions/2010/1/29/PDF,2010-1-29,https://www.al-akhbar.com/PDF_Files/1031/alakh...


In [82]:
df.pdf_link.isna().value_counts()

True     1945
False     983
Name: pdf_link, dtype: int64

In [2]:
full_df = pd.concat([df1,df2,df3], ignore_index=True)
full_df.drop(columns=['Unnamed: 0'], inplace= True)

NameError: name 'df1' is not defined

In [84]:
full_df.drop_duplicates(inplace=True)

In [85]:
full_df.pdf_link.isna().value_counts()

False    1910
True      159
Name: pdf_link, dtype: int64

In [91]:
df.loc[df['pdf_link'].isna(), 'pdf_link'] = full_df['pdf_link']

In [95]:
df.loc[df['pdf_link'].isna(), 'pdf_link'] = df['url'].apply(get_pdf)

tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tama

In [107]:
df.pdf_link.isna().value_counts()

False    2870
True       58
Name: pdf_link, dtype: int64

In [108]:
df.loc[df['pdf_link'].isna(), 'pdf_link'] = df.loc[df['pdf_link'].isna(), 'url'].apply(get_pdf)

tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam
tamam


In [110]:
df.to_csv('Full_df.csv')

In [3]:
df = pd.read_csv('Full_df.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df

,issue,url,date,pdf_link
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...
1,العدد 3923,https://www.al-akhbar.com/Editions/2019/12/3/PDF,2019-12-3,https://www.al-akhbar.com/PDF_Files/3885/alakh...
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...
...,...,...,...,...
2923,العدد 1028,https://www.al-akhbar.com/Editions/2010/1/26/PDF,2010-1-26,https://www.al-akhbar.com/PDF_Files/1028/alakh...
2924,العدد 1029,https://www.al-akhbar.com/Editions/2010/1/27/PDF,2010-1-27,https://www.al-akhbar.com/PDF_Files/1029/alakh...
2925,العدد 1030,https://www.al-akhbar.com/Editions/2010/1/28/PDF,2010-1-28,https://www.al-akhbar.com/PDF_Files/1030/alakh...
2926,العدد 1031,https://www.al-akhbar.com/Editions/2010/1/29/PDF,2010-1-29,https://www.al-akhbar.com/PDF_Files/1031/alakh...


In [69]:
os.chdir('/home/noahdarwich/code/NoahDarwich/Leb_analysis/Notebooks/')

In [4]:
os.getcwd()

'/home/noahdarwich/code/NoahDarwich/Leb_analysis/Notebooks'

In [5]:
def save_pdfs(df):
    
    unsaved = []
    df = df.dropna()
    for i, j in df.iterrows():
        
        name = j['date']+'.pdf'
        url = j['pdf_link']
        
        response = requests.get(url)
        
        try:
            os.chdir('/mnt/d')
            with open(name, 'wb') as file:
                file.write(response.content)
                print('tamam')
        except:
            unsaved.append(name)
            print('No')
    return unsaved

In [6]:
unsaved = save_pdfs(df)

No
No
No
No


KeyboardInterrupt: 

In [53]:
import tempfile 
from oauth2client.service_account import ServiceAccountCredentials

In [66]:
name = df.date[0]+'.pdf'
link = df.pdf_link[0]
response = requests.get(link)

In [67]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-61adff20f07d.json', scopes=['https://www.googleapis.com/auth/drive'])

service = build('drive', 'v3', credentials=creds)

#file_metadata = {'name': name}



#file = service.files().create(body=file_metadata, media_body=media,
#                                    fields='id').execute()
#print(F'File ID: {file.get("id")}')

In [68]:
folder_metadata = {
    'name': 'Al_Akhbar_2010_2019',
    'mimeType': 'application/vnd.google-apps.folder'
}
folder = service.files().create(body=folder_metadata, fields='id').execute()
folder_id = folder.get('id')


In [69]:
file = tempfile.NamedTemporaryFile()
file.write(response.content)
file.seek(0)

file_metadata = {'name': name,'parents':[folder_id]}
media = MediaFileUpload(file.name,
                        mimetype='application/pdf')

file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

In [ ]:
# def save_article_to_drive(article_text, file_name):
#     try:
#       # authenticate with Google Drive
#       credentials = Credentials.from_authorized_user_info()
#       service = build('drive', 'v3', credentials=credentials)

#       # create the file on Google Drive
#       file_metadata = {'name': file_name, 'mimeType': 'text/plain'}
#       media = MediaIoBaseUpload(io.StringIO(article_text), mimetype='text/plain',chunksize=1024*1024, resumable=True)
#       file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
#       print(f'File ID: {file.get(/id\)}')
      
#     except HttpError as error:
#         print(f'An error occurred: {error}')
#         file = None
#     return file

In [ ]:
# scrape the website for PDF links

# download each PDF and save it to Google Drive

text_ls = [] 
for link in pdf_links:
    # download the PDF
    pdf_url = link['href']
    
    pdf_content = pdf_response.content
    # extract the text from the PDF
    text = extract_text_from_pdf(pdf_content)
    text_ls.append(text)
    
print(soup)

In [ ]:
# def extract_text_from_pdf(pdf_content):
#   resource_manager = PDFResourceManager()
#   fake_file_handle = io.StringIO()
#   converter = TextConverter(resource_manager, fake_file_handle)
#   page_interpreter = PDFPageInterpreter(resource_manager, converter)

#   with io.BytesIO(pdf_content) as fh:
#     for page in PDFPage.get_pages(fh,
#                                   caching=True,
#                                   check_extractable=True):
#       page_interpreter.process_page(page)

#     text = fake_file_handle.getvalue()

#   # close open handles
#   converter.close()
#   fake_file_handle.close()

#   return text

In [5]:

 
  
# "# search the text for keywords
# "keywords = ['keyword1', 'keyword2', 'keyword3']
# "for keyword in keywords:
# "  if re.search(keyword, text, re.IGNORECASE):
# "    # keyword found, save the article
# "    file_name = f'{keyword}_article.txt'
# "    save_article_to_drive(text, file_name)
# "    break"